**A NOVEL APPROACH TO MALAYALAM SPEECH-TO-TEXT AND TEXT-TO-ENGLISH
TRANSLATION**

The provided code creates a speech-to-text and translation application using Tkinter and various libraries. Here's a breakdown of the code and potential improvements:

**Code Breakdown:**

- **Imports:**
    - Tkinter for GUI elements.
    - `speech_recognition` for speech recognition.
    - `threading` for running speech recognition in a separate thread.
    - `indicnlp` and `ctranslate2` for Malayalam text processing and translation.
    - `sentencepiece` for sentence tokenization.
    - `matplotlib` (not directly used, potentially a leftover import).
- **`translate_text` Placeholder:** This function needs to be implemented to handle the translation logic based on your chosen translation library.
- **`add_token` Function:** Adds language tags to the sentence.
- **`preprocess_sentence` Function:**
    - Normalizes Malayalam text.
    - Transliterates to Hindi.
    - Splits into sentences and tokens.
    - Handles sentences exceeding a maximum length.
    - Uses `ctranslate2` for translation.
    - Returns the translated text.
- **`EloquenSpeakerApp` Class:**
    - Initializes the Tkinter window, recognizer, and microphone.
    - Sets up the GUI elements:
        - Title and geometry for the window.
        - Labels for instructions and text boxes.
        - Buttons for start recording and exit.
    - **`start_recording` Function:**
        - Starts recording in a separate thread.
        - Updates the UI during different stages.
    - **`update_transcription` Function:**
        - Updates the transcription text box with recognized text and calls the translation function.
        - Updates the translation text box.
        - Re-enables the start recording button.
    - **`recognize_speech` Function:**
        - Adjusts for ambient noise.
        - Listens for audio input.
        - Uses `recognizer.recognize_google` to convert audio to text.
        - Handles potential errors (UnknownValueError, RequestError).
    - **`exit_application` Function:**
        - Quits the Tkinter application.

**Potential Improvements:**

- **Error Handling:** Consider adding more specific error handling for different translation issues.
- **Translation Library Integration:** Implement the `translate_text` function based on your preferred translation library (e.g., `googletrans`).
- **User Control:** Allow users to choose the translation language from the GUI.
- **Stop Recording Functionality:** Add a button or hotkey to stop recording.
- **Progress Bar:** Consider adding a progress bar or visual indicator while translating.
- **Clear Text Boxes:** Allow users to clear the existing text in the boxes before recording.

**Running the Code:**

1. Install required libraries using `pip install tkinter speech_recognition indicnlp ctranslate2 sentencepiece` (if not already installed).
2. Complete the implementation of the `translate_text` function for your chosen translation library.
3. Make sure you have the necessary model files for the translation process (if applicable).
4. Run the script using `python your_script_name.py`.

This enhanced code provides a solid foundation for a speech-to-text and translation application in Python. Feel free to tailor it further based on your specific requirements and desired functionalities.


In [ ]:
!pip install indicnlp

In [ ]:
!pip install indic-nlp-library

In [ ]:
!pip install ctranslate2

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install speechrecognition

In [ ]:
!pip install tk

In [17]:
import tkinter as tk
from tkinter import ttk, scrolledtext
import speech_recognition as sr
import threading
import os
from indicnlp.tokenize import indic_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.transliterate import unicode_transliterate
import ctranslate2
import sentencepiece as spm
import matplotlib
matplotlib.use('Agg')
# Placeholder for the translate_text function
def translate_text(text):
    model_dir = "C:/Users/user/Downloads/final_model(1)/final_model"
    malayalam_sentence = text

    translated_sentence = translate_sentence(malayalam_sentence, model_dir)
    print("Translated sentence:", translated_sentence)
    return "Translated Text: " + translated_sentence

def add_token(sent: str):

    return "mal_Mlym" + " " + "eng_Latn" + " " + sent

def preprocess_sentence(sentence: str,sp_src,model_dir) -> str:

    normfactory = IndicNormalizerFactory()
    normalizer = normfactory.get_normalizer("ml")
    sent=sentence
    xliterator = unicode_transliterate.UnicodeIndicTransliterator()
    processed_sent = xliterator.transliterate(" ".join(indic_tokenize.trivial_tokenize(normalizer.normalize(sent.strip()), "ml")),"ml",
                "hi",
            ).replace(" ् ", "्")
    sents= [" ".join(sp_src.encode(sent, out_type=str)) for sent in [processed_sent]]

    tagged_sents = []
    for sent in sents:
        tagged_sent = add_token(sent.strip())
        tagged_sents.append(tagged_sent)
    MAX_SEQ_LEN = 256
    new_sents = []

    for sent in tagged_sents:
        words = sent.split()
        num_words = len(words)
        if num_words > MAX_SEQ_LEN:
            print_str = " ".join(words[:5]) + " .... " + " ".join(words[-5:])
            sent = " ".join(words[:MAX_SEQ_LEN])
            print(
                f"WARNING: Sentence {print_str} truncated to 256 tokens as it exceeds maximum length limit"
            )

        new_sents.append(sent)

    translator = ctranslate2.Translator(model_dir, device="cpu")
    tokenized_sents = [x.strip().split(" ") for x in new_sents]
    translations = translator.translate_batch(
            tokenized_sents,
            max_batch_size=9216,
            batch_type="tokens",
            max_input_length=160,
            max_decoding_length=256,
            beam_size=5,
        )
    translations = [" ".join(x.hypotheses[0]) for x in translations]
    for i in range(len(translations)):
        translations[i] = translations[i].replace(" ", '').replace("▁", " ").strip()

    return translations[0]

def translate_sentence(sentence: str, model_dir: str) :

    sp_src = spm.SentencePieceProcessor(model_file=os.path.join(model_dir, "vocab", "model.SRC"))

    preprocessed_sentence = preprocess_sentence(sentence,sp_src,model_dir)

    return preprocessed_sentence

class EloquentSpeakerApp:
    def __init__(self, root):
        self.root = root
        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()
        self.setup_ui()

    def setup_ui(self):
        self.root.title("Eloquent Speaker - Speech to Text and Translation")
        self.root.geometry("1000x700")  # Increased window size

        # UI Styling
        self.root.config(bg="#1E2630")

        ttk.Label(self.root, text="Eloquent Speaker", font=("Arial", 28, "bold"), background="#1E2630", foreground="#FFD700").pack(pady=20)
        ttk.Label(self.root, text="Click 'Start Recording' and speak in Malayalam.", font=("Arial", 14), background="#1E2630", foreground="white").pack(pady=10)

        self.start_button = ttk.Button(self.root, text="Start Recording", command=self.start_recording, width=20)
        self.start_button.pack(pady=10)

        self.transcription_box = scrolledtext.ScrolledText(self.root, wrap=tk.WORD, width=80, height=8, font=("Arial", 14), bg="white")
        self.transcription_box.pack(pady=10)
        self.transcription_box.insert(tk.END, "Original Malayalam Text:\n")
        self.transcription_box.config(state=tk.DISABLED)

        self.translation_box = scrolledtext.ScrolledText(self.root, wrap=tk.WORD, width=80, height=8, font=("Arial", 14), bg="white")
        self.translation_box.pack(pady=10)
        self.translation_box.insert(tk.END, "Translated English Text:\n")
        self.translation_box.config(state=tk.DISABLED)

        self.exit_button = ttk.Button(self.root, text="Exit", command=self.exit_application, width=20)
        self.exit_button.pack(pady=20)

    def start_recording(self):

        self.is_recording = True
        self.start_button['state'] = 'disabled'
        self.transcription_box.config(state=tk.NORMAL)
        self.transcription_box.delete(1.0, tk.END)
        self.transcription_box.insert(tk.END, "Please wait...\n")
        self.transcription_box.config(state=tk.DISABLED)
        threading.Thread(target=self.recognize_speech).start()

    def update_transcription(self, text):
        self.transcription_box.config(state=tk.NORMAL)
        self.transcription_box.delete(1.0, tk.END)
        self.transcription_box.insert(tk.END, f"Transcribed Text:\n{text}")
        self.transcription_box.config(state=tk.DISABLED)

        # Assuming translate_text function exists and returns the translation
        translated_text = translate_text(text)
        self.translation_box.config(state=tk.NORMAL)
        self.translation_box.delete(1.0, tk.END)
        self.translation_box.delete(0.0, tk.END)
        self.translation_box.insert(tk.END, translated_text)
        self.translation_box.config(state=tk.DISABLED)
        self.start_button['state'] = 'enable'

    def recognize_speech(self):
        with self.microphone as source:
            self.recognizer.adjust_for_ambient_noise(source, duration=5)
            self.transcription_box.config(state=tk.NORMAL)
            self.transcription_box.delete(1.0, tk.END)
            self.transcription_box.insert(tk.END, "Listening...\n")
            self.transcription_box.config(state=tk.DISABLED)
            audio = self.recognizer.listen(source)
            try:
                #print(1)
                transcription = self.recognizer.recognize_google(audio, language="ml-IN")
                print(transcription)
                self.transcription_box.config(state=tk.NORMAL)
                self.transcription_box.delete(1.0, tk.END)
                self.transcription_box.insert(tk.END, f"Transcribed Text:\n{transcription}")
                self.update_transcription(f"{transcription}")
            except sr.UnknownValueError:
                self.update_transcription("Could not understand audio.")
            except sr.RequestError as e:
                self.update_transcription(f"Could not request results; {e}.")

    def exit_application(self):
        self.root.quit()

if __name__ == "__main__":
    root = tk.Tk()
    app = EloquentSpeakerApp(root)
    root.mainloop()


TclError: no display name and no $DISPLAY environment variable